# <font color='blue'>  Sec 10. Prompt Chain in Python </font>


#### <font color='red'> Content: Automated Claim Triage in Insurance Industry </font>
In this hands-on exercise, you will <font color='red'>  build a prompt chain that extracts key fields from free-form auto-claim reports, assesses damage severity, and routes each claim to one of several queues </font> — `glass`, `fast_track`, `material_damage`, or `total_loss` — with code-based gate checks at every step. 

### Outline:

    0. Setup
    1. Sample FNOL (First Notice of Loss) Texts
    2. Stage I: Information Extraction
    3. Stage II: Severity Assessment
    4. Stage III: Queue Routing
    5. Review Outputs

## 0. Setup

In [9]:
from openai import OpenAI
from IPython.display import Markdown, display
import os

import json

with open('api_key.json', 'r') as file:
    api_keys = json.load(file)
    openai_key = api_keys["openai"]

# print(openai_key)
client = OpenAI(api_key=openai_key)

from utils import get_completion
from utils_prompt_chain import (
    ClaimInformation, 
    assess_severity,
    extract_claim_info,
    gate1_validate_claim_info,
    gate2_cost_range_ok,
    route_claim,
)


## 1. Sample FNOL (First Notice of Loss) Texts

Let's define a few sample First Notice of Loss (FNOL) texts to process through our chain.

In [2]:
# Define sample FNOL texts

sample_fnols = [
    """
    Claim ID: C001
    Customer: John Smith
    Vehicle: 2018 Toyota Camry
    Incident: While driving on the highway, a rock hit my windshield and caused a small chip
    about the size of a quarter. No other damage was observed.
    """,
    """
    Claim ID: C002
    Customer: Sarah Johnson
    Vehicle: 2020 Honda Civic
    Incident: I was parked at the grocery store and returned to find someone had hit my car and
    dented the rear bumper and taillight. The taillight is broken and the bumper has a large dent.
    """,
    """
    Claim ID: C003
    Customer: Michael Rodriguez
    Vehicle: 2022 Ford F-150
    Incident: I was involved in a serious collision at an intersection. The front of my truck is
    severely damaged, including the hood, bumper, radiator, and engine compartment. The airbags
    deployed and the vehicle is not drivable.
    """,
    """
    Claim ID: C004
    Customer: Emma Williams
    Vehicle: 2019 Subaru Outback
    Incident: My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and
    trunk. The side mirrors were also damaged and one window has a small crack.
    """,
    """
    Claim ID: C005
    Customer: David Brown
    Vehicle: 2021 Tesla Model 3
    Incident: Someone keyed my car in the parking lot. There are deep scratches along both doors
    on the driver's side.
    """,
]

## 2. Stage I: Information Extraction

In this stage, we'll create a prompt that extracts structured information from free-form FNOL text.

In [3]:
# Define a system prompt for information extraction according to the provided ClaimInformation class

info_extraction_system_prompt = """
You are an data extractor AI in insurance company. Your task is to extract key information from First Notice of Loss (FNOL) reports.

Format your response as a valid JSON object with the following keys:
- claim_id (str): The claim ID
- name (str): The customer's full name 
- vehicle (str): The make, model, year.
- loss_desc (str): The damage descriptions on the vehicle.
- damage_area: (list[Literal[str]]): The list of damage areas in the vehicle.
- etc.

You need to read the claim, and extract the related info in the response. Note:
- The keys in the response JSON has the right data type.
- Explain why damage areas are listed from Incident descrption.
- For damage areas, if there are "fornt hood", "rear bumper", parse to "hood", "bumper".
- For any plur noun (with s, es, ies etc), take singular. For exmaple, "doors", take "door".
- Judge damage severity, and explain why you provided the judgement.

Only respond with the JSON object, nothing else.
"""

In [4]:
# Run the claim extraction function on the sample FNOLs
# No updates needed in this cell

extracted_claim_info_items = [
    extract_claim_info(client, fnol_text, info_extraction_system_prompt) for fnol_text in sample_fnols
]
extracted_claim_info_items

[ClaimInformation(claim_id='C001', name='John Smith', vehicle='2018 Toyota Camry', loss_desc='Small chip on windshield caused by rock impact', damage_area=['windshield']),
 ClaimInformation(claim_id='C002', name='Sarah Johnson', vehicle='2020 Honda Civic', loss_desc='The taillight is broken and the bumper has a large dent.', damage_area=['taillight', 'bumper']),
 ClaimInformation(claim_id='C003', name='Michael Rodriguez', vehicle='2022 Ford F-150', loss_desc='The front of the truck is severely damaged, including the hood, bumper, radiator, and engine compartment. Airbags deployed and vehicle is not drivable.', damage_area=['hood', 'bumper', 'radiator', 'engine compartment']),
 ClaimInformation(claim_id='C004', name='Emma Williams', vehicle='2019 Subaru Outback', loss_desc='Multiple dents on the hood, roof, trunk; damage to side mirror; small crack on window', damage_area=['hood', 'roof', 'trunk', 'mirror', 'window']),
 ClaimInformation(claim_id='C005', name='David Brown', vehicle='2021

## 3. Stage II: Severity Assessment
In this stage, we'll assess the severity of the damage based on the extracted information.

Note, our carrier applies the following heuristics:
- Minor damage: Small dents, scratches, glass chips (cost range: $100-$1,000)
- Moderate damage: Single panel damage, bumper replacement, door damage (cost range: $1,000-$5,000)
- Major damage: Structural damage, multiple panel replacement, engine/drivetrain issues, total loss candidates (cost range: $5,000-$50,000)

In this example we will let the LLM estimate the cost, though in production we would want a more accurate estimate, e.g. querying a database of repair costs.

In [5]:
severity_assessment_system_prompt = f"""
You are an auto insurance damage assessor. Your task is to evaluate the severity of vehicle damage and estimate repair costs.

The claims are given in {extracted_claim_info_items}.

The report follows the guidelines:
- First read each claim to understand damage_area.
- Understand loss_desc (description).
- Determine severity of each vehicle damage to SeverityAssessment.severity in ("Minor," "Moderate," "Major").
- According to damage_area, loss_desc, severity, give estimate cost of the repair.
- The severity and typical costs are usually: Minor ($100-$1000), Moderate ($1000-$5000), Major ($5000-$50000).
- Take the estimated cost to SeverityAssessment.est_cost as float.

Only respond with the JSON object, nothing else.
"""

In [6]:
# Run the claim extraction function on the sample data
# No updates needed in this cell

severity_assessment_items = [
    assess_severity(client, severity_assessment_system_prompt, item) for item in extracted_claim_info_items
]

severity_assessment_items

[SeverityAssessment(severity='Minor', est_cost=300),
 SeverityAssessment(severity='Moderate', est_cost=2500),
 SeverityAssessment(severity='Major', est_cost=25000),
 SeverityAssessment(severity='Moderate', est_cost=3000),
 SeverityAssessment(severity='Moderate', est_cost=2000)]

## 4. Stage III: Queue Routing

In this stage, we'll route the claim to the appropriate queue based on severity and damage area.

Use these routing rules:
- `glass` queue: For **Minor** damage involving ONLY glass (windshield, windows)
- `fast_track` queue: For other **Minor** damage
- `material_damage` queue: For all **Moderate** damage
- `total_loss` queue: For all **Major** damage

There are priotiry levels:
* Priority 1: (highest), safety issue, customer standard
* Priority 2: significant but container damage, vechicle drivable
* Priority 3: standard claim
* Priority 4: minor issue, not mobility impact
* Priority 5: (lowest), cosmetric only, not functional impact

In [21]:
queue_routing_system_prompt = """
    You are an auto insurance claim routing specialist. Your task is to determine the appropriate processing queue for each claim.
    
    Based on the claim information and severity assessment, analyze severity, damage areas, and assign:
    - a queue
    - a priority from 1 (highest) to 5 (lowest) based on the overall situation described.
    
    Each claim should be assigned to the following queues: 'glass', 'fast_track', 'material_damage', or 'total_loss' using the guidence:
    - Minor damage involving ONLY glass goes to 'glass'. 
    - Other Minor damage goes to 'fast_track'. 
    - Moderate damage goes to 'material_damage'. 
    - Major damage goes to 'total_loss'.

    Each claim should be also assigned to a prority using the rule:
    - Priority 1: (highest), safety issue, customer standard
    - Priority 2: significant but container damage, vechicle drivable
    - Priority 3: standard claim
    - Priority 4: minor issue, not mobility impact
    - Priority 5: (lowest), cosmetric only, not functional impact

    Format the response with JSON object:
    - claim_id: str
    - queue: one of 'glass', 'fast_track', 'material_damage', or 'total_loss'
    - priority: 1-5. 1 has highest priority, and strongly suggest to fix.
    
    Only respond with the JSON object, nothing else.
"""

In [22]:
# Run the routing function on the sample data
# No updates needed in this cell

routed_claim_items = [
    route_claim(client,
                queue_routing_system_prompt,
                claim,
                severity_assessment
    )
    for claim, severity_assessment in zip(
        extracted_claim_info_items, severity_assessment_items
    )
]

routed_claim_items

[ClaimRouting(claim_id='C001', queue='glass'),
 ClaimRouting(claim_id='C002', queue='material_damage'),
 ClaimRouting(claim_id='C003', queue='total_loss'),
 ClaimRouting(claim_id='C004', queue='material_damage'),
 ClaimRouting(claim_id='C005', queue='material_damage')]

## 5. Review Outputs

Let's put our data into a pandas dataframe for easier analysis.

In [23]:
# No updates needed in this cell

import pandas as pd

records = []
for claim, severity_assessment, routed_claim in zip(
    extracted_claim_info_items, severity_assessment_items, routed_claim_items
):
    record = {}
    record.update(claim)
    record.update(severity_assessment)
    record.update(routed_claim)
    records.append(record)


# Show the entire dataframe since it is not too large
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(records)

df

,claim_id,name,vehicle,loss_desc,damage_area,severity,est_cost,queue
0,C001,John Smith,2018 Toyota Camry,Small chip on windshield caused by rock impact,[windshield],Minor,300,glass
1,C002,Sarah Johnson,2020 Honda Civic,The taillight is broken and the bumper has a large dent.,"[taillight, bumper]",Moderate,2500,material_damage
2,C003,Michael Rodriguez,2022 Ford F-150,"The front of the truck is severely damaged, including the hood, bumper, radiator, and engine compartment. Airbags deployed and vehicle is not drivable.","[hood, bumper, radiator, engine compartment]",Major,25000,total_loss
3,C004,Emma Williams,2019 Subaru Outback,"Multiple dents on the hood, roof, trunk; damage to side mirror; small crack on window","[hood, roof, trunk, mirror, window]",Moderate,3000,material_damage
4,C005,David Brown,2021 Tesla Model 3,Deep scratches along both doors on the driver's side,[door],Moderate,2000,material_damage
